Para replicar el referenciamiento de ImageNet en tu propio dataset de imágenes almacenado en Amazon S3 y ejecutar la transferencia de aprendizaje con ResNet, debes seguir varios pasos. Aquí tienes una guía general sobre cómo hacerlo:


In [2]:
# Librerías estándar
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from io import BytesIO

# TensorFlow y Keras
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import (Conv2D, Dense, Dropout, Flatten, GlobalAveragePooling2D, MaxPooling2D, Reshape)
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

# Otras librerías
import os
import boto3
from PIL import Image
from sklearn.metrics import (accuracy_score, auc, average_precision_score, classification_report, 
                             confusion_matrix, precision_score, recall_score)
from sklearn.model_selection import train_test_split

2023-10-09 11:50:33.752075: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


# Entrenamiento Animal y No Animal

In [2]:
# Carga la base de datos de Train y Test
train = pd.read_pickle('trainingAnimal.pkl')
test = pd.read_pickle('testingAnimal.pkl')

In [3]:
X_train = train['Imagen']
y_train = train['Animal']
X_test = test['Imagen']
y_test = test['Animal']

In [4]:
# Convierte los datos de entrenamiento y etiquetas en tensores de TensorFlow
X_train_tf = tf.convert_to_tensor(np.array([img_to_array(img) for img in X_train]))
X_test_tf = tf.convert_to_tensor(np.array([img_to_array(img) for img in X_test]))

## Descarga del modelo ResNet50 pre-entrenado:
Descarga el modelo ResNet50 pre-entrenado con pesos de ImageNet. Puedes hacerlo utilizando TensorFlow o Keras.

In [7]:
input_shape = (150, 150, 3)

In [8]:
# Carga el modelo ResNet50 preentrenado con pesos de ImageNet 
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

94765736/94765736 [==============================] - 12s 0us/step


## Generador de datos:

Utiliza un generador de datos de Keras para cargar y preprocesar tus imágenes desde S3. Debes proporcionar la ruta a tus imágenes en S3 y etiquetas correspondientes. Aquí un ejemplo de cómo configurar un generador de datos:

In [9]:
# Número de clases en tu conjunto de datos
num_classes = len(y_train.unique())

In [10]:
# Tamaño del lote (batch size) que deseas utilizar durante el entrenamiento
batch_size = 32

## Entrenamiento del modelo:

Añade capas personalizadas en la parte superior del modelo VGG16 y entrena el modelo en tus datos utilizando el generador de datos. Asegúrate de congelar las capas base de ResNet para que no se actualicen durante el entrenamiento.

In [11]:
# Agregar capas personalizadas en la parte superior del modelo base
x = Flatten()(base_model.output) #Flatten output to 1 dimension
x = Dense(1024,activation='relu')(x) #Añade una layer con Relu activation
x = Dropout(0.2)(x) #Añade un dropout rate de 0.2
predictions = Dense(1, activation = 'sigmoid')(x)

In [12]:
# Crear el modelo final
model = Model(inputs=base_model.input, outputs=predictions)

In [13]:
# Congelar las capas del modelo base para el transfer learning
for layer in base_model.layers:
    layer.trainable = False

In [14]:
# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
# Define the EarlyStopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=5,           # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored quantity
)

In [16]:
# Entrena el modelo con los datos de entrenamiento
model.fit(X_train_tf, y_train, epochs=30, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/30
119/119 [==============================] - 152s 1s/step - loss: 2.1522 - accuracy: 0.9505 - val_loss: 2.4049 - val_accuracy: 0.9663
Epoch 2/30
119/119 [==============================] - 144s 1s/step - loss: 0.7180 - accuracy: 0.9797 - val_loss: 2.4509 - val_accuracy: 0.9663
Epoch 3/30
119/119 [==============================] - 147s 1s/step - loss: 0.5401 - accuracy: 0.9784 - val_loss: 2.0228 - val_accuracy: 0.9611
Epoch 4/30
119/119 [==============================] - 144s 1s/step - loss: 0.2824 - accuracy: 0.9860 - val_loss: 2.1384 - val_accuracy: 0.9611
Epoch 5/30
119/119 [==============================] - 144s 1s/step - loss: 0.2759 - accuracy: 0.9863 - val_loss: 1.3570 - val_accuracy: 0.9463
Epoch 6/30
119/119 [==============================] - 144s 1s/step - loss: 0.1690 - accuracy: 0.9881 - val_loss: 1.3414 - val_accuracy: 0.9611
Epoch 7/30
119/119 [==============================] - 144s 1s/step - loss: 0.1566 - accuracy: 0.9897 - val_loss: 1.0647 - val_accuracy: 0.9653

In [21]:
# Guardar el modelo entrenado
model.save('modeloAnimalRN50.h5')

In [ ]:
y_proba = model.predict(X_test_tf)
y_pred = (y_proba >= 0.5).astype(int)

In [ ]:
# Calculamos Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Calculamos Recall
recall = recall_score(y_test, y_pred, pos_label=1, average='binary')
print(f"Recall: {recall}")

# Calculamos Specificity
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
print(f"Specificity: {specificity}")

Accuracy: 0.9713563605728728
Recall: 0.9838709677419355
Specificity: 0.9576719576719577


In [22]:
# Calcular average precision
ap = average_precision_score(y_test, y_proba)

print("Average Precision (AP):", ap)

Average Precision (AP): 0.9829587558124374


In [23]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.96      0.97       567
           1       0.96      0.98      0.97       620

    accuracy                           0.97      1187
   macro avg       0.97      0.97      0.97      1187
weighted avg       0.97      0.97      0.97      1187



# Entrenamiento Guanaco y No Guanaco

In [72]:
# Carga la base de datos de Train y Test
train = pd.read_pickle('trainingGuanaco.pkl')
test = pd.read_pickle('testingGuanaco.pkl')

In [73]:
X_train = train['Imagen']
y_train = train['Guanaco']
X_test = test['Imagen']
y_test = test['Guanaco']

In [74]:
# Convierte los datos de entrenamiento y etiquetas en tensores de TensorFlow
X_train_tf = tf.convert_to_tensor(np.array([img_to_array(img) for img in X_train]))
X_test_tf = tf.convert_to_tensor(np.array([img_to_array(img) for img in X_test]))

## Balanceo

In [75]:
y_train.value_counts()

Guanaco
True     1187
False     541
Name: count, dtype: int64

La base esta desbalanceada ya que existen muchos mas guanacos que otros animales. Para resolver esto se somete al resto de los animales a tecnicas de *Data Augmentation*

Data Augmentation: make training set larger by applying transformations. More information to learn from.
- Brightness and Contrast adjustments
- Rotations
- Gaussian noise
- Mirroring

In [76]:
def data_augmentation(image_tensor):

    # Convierte el tensor de imagen a una imagen TensorFlow
    image = tf.convert_to_tensor(image_tensor, dtype=tf.float32)

    # Brightness and Contrast adjustments
    if np.random.rand() < 0.8:
        image = tf.image.adjust_brightness(image, delta=0.2)  # Cambiar el brillo
        image = tf.image.adjust_contrast(image, contrast_factor=1.2)  # Cambiar el contraste

    # Rotations
    if np.random.rand() < 0.7:
        degrees = np.random.uniform(-10, 10)  # Rotación aleatoria entre -10 y 10 grados
        degrees = int(round(degrees))  # Redondea los grados a un entero
        image = tf.image.rot90(image, k=degrees // 90)  # Rotar la imagen

    # Gaussian noise
    if np.random.rand() < 0.2:
        noise = tf.random.normal(shape=tf.shape(image), mean=0.0, stddev=0.1)
        image = image + noise

    # Mirroring (flip horizontal)
    if np.random.rand() < 0.5:
        image = tf.image.flip_left_right(image)

    # Convierte la imagen aumentada de nuevo a un tensor
    augmented_image_tensor = tf.convert_to_tensor(image.numpy(), dtype=tf.float32)

    return augmented_image_tensor

In [77]:
# Crea una nueva lista para almacenar los tensores de imágenes aumentados
imagen_tensor_aumentada = []

# Itera a través de las filas del DataFrame y aplica la función de aumento de datos
for index, row in train.iterrows():
    if np.random.rand() < 0.38:
        imagen_tensor = row['Imagen']
        imagen_aumentada = data_augmentation(imagen_tensor)
        imagen_tensor_aumentada.append(imagen_aumentada)

In [78]:
train_augmentation = pd.DataFrame(columns=['Imagen', 'Guanaco'])
train_augmentation['Imagen'] = imagen_tensor_aumentada
train_augmentation['Guanaco'] = False

In [79]:
train_augmentation= pd.concat([train, train_augmentation], ignore_index=True)

In [80]:
train_augmentation['Guanaco'].value_counts()

Guanaco
False    1200
True     1187
Name: count, dtype: int64

In [81]:
X_train = train_augmentation['Imagen']
y_train = train_augmentation['Guanaco']

In [82]:
X_train_tf = tf.convert_to_tensor(np.array([img_to_array(img) for img in X_train]))

## Descarga del modelo VGG16 pre-entrenado:
Descarga el modelo VGG16 pre-entrenado con pesos de ImageNet. Puedes hacerlo utilizando TensorFlow o Keras.

In [83]:
input_shape = (150, 150, 3)

In [84]:
# Carga el modelo ResNet50 preentrenado con pesos de ImageNet 
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

## Entrenamiento del modelo:

Añade capas personalizadas en la parte superior del modelo VGG16 y entrena el modelo en tus datos utilizando el generador de datos. Asegúrate de congelar las capas base de VGG16 para que no se actualicen durante el entrenamiento.

In [85]:
# Agregar capas personalizadas en la parte superior del modelo base
x = Flatten()(base_model.output)  # Flatten output to 1 dimension
# x = Dense(1024, activation='relu')(x)  # Añade una capa con Relu activation
# x = Dropout(0.2)(x)  # Añade un dropout rate de 0.2

# # Agregar más capas densas
# x = Dense(512, activation='relu')(x)
# x = Dropout(0.3)(x)

# # Agregar más capas densas
# x = Dense(256, activation='relu')(x)
# x = Dropout(0.2)(x)

# Capa de salida
predictions = Dense(1, activation='sigmoid')(x)

In [86]:
# Crear el modelo final
model = Model(inputs=base_model.input, outputs=predictions)

In [87]:
# Congelar las capas del modelo base para el transfer learning
for layer in base_model.layers:
    layer.trainable = False

In [88]:
# Define the EarlyStopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=5,           # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored quantity
)

In [89]:
# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy'])

In [90]:
# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train_tf, y_train, epochs=30, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/30
60/60 [==============================] - 142s 2s/step - loss: 0.9313 - accuracy: 0.7260 - val_loss: 1.2911 - val_accuracy: 0.7197
Epoch 2/30
60/60 [==============================] - 154s 3s/step - loss: 0.4006 - accuracy: 0.8607 - val_loss: 1.0035 - val_accuracy: 0.7992
Epoch 3/30
60/60 [==============================] - 167s 3s/step - loss: 0.2827 - accuracy: 0.9005 - val_loss: 1.2139 - val_accuracy: 0.7741
Epoch 4/30
60/60 [==============================] - 151s 3s/step - loss: 0.1478 - accuracy: 0.9471 - val_loss: 1.0425 - val_accuracy: 0.7887
Epoch 5/30
60/60 [==============================] - 165s 3s/step - loss: 0.1113 - accuracy: 0.9649 - val_loss: 1.6127 - val_accuracy: 0.7280
Epoch 6/30
60/60 [==============================] - 149s 2s/step - loss: 0.1187 - accuracy: 0.9591 - val_loss: 1.6764 - val_accuracy: 0.7218
Epoch 7/30
60/60 [==============================] - 122s 2s/step - loss: 0.0992 - accuracy: 0.9680 - val_loss: 1.4536 - val_accuracy: 0.7448


In [67]:
# Guardar el modelo entrenado
model.save('modeloGuanacoRN50.h5')

In [91]:
y_proba = model.predict(X_test_tf)
y_pred = (y_proba >= 0.5).astype(int)

14/14 [==============================] - 24s 2s/step


In [92]:
# Calculamos Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Calculamos Recall
recall = recall_score(y_test, y_pred, pos_label=1, average='binary')
print(f"Recall: {recall}")

# Calculamos Precision
precision = precision_score(y_test, y_pred, pos_label=1, average='binary')
print(f"Precision: {precision}")

# Calculamos Specificity
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
print(f"Specificity: {specificity}")

# Accuracy: 0.8290993071593533
# Recall: 0.8619528619528619
# Precision: 0.8858131487889274
# Specificity: 0.7573529411764706

Accuracy: 0.789838337182448
Recall: 0.761744966442953
Precision: 0.9190283400809717
Specificity: 0.8518518518518519


In [70]:
# Calcular average precision
ap = average_precision_score(y_test, y_proba)

print("Average Precision (AP):", ap)

Average Precision (AP): 0.8960072957860057


In [71]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.72      0.76      0.74       136
        True       0.89      0.86      0.87       297

    accuracy                           0.83       433
   macro avg       0.80      0.81      0.80       433
weighted avg       0.83      0.83      0.83       433



In [ ]:
(y_pred == 0).sum() / len(y_pred)

0.7780320366132724

# Entrenamiento Categoria Especie

In [ ]:
# Carga la base de datos de Train y Test
train = pd.read_pickle('trainingCategoria.pkl')
test = pd.read_pickle('testingCategoria.pkl')

In [ ]:
X_train = train['Imagen']
y_train = train['Categoria']
X_test = test['Imagen']
y_test = test['Categoria']

In [ ]:
# Convierte los datos de entrenamiento y etiquetas en tensores de TensorFlow
X_train_tf = tf.convert_to_tensor(np.array([img_to_array(img) for img in X_train]))
X_test_tf = tf.convert_to_tensor(np.array([img_to_array(img) for img in X_test]))